<a href="https://colab.research.google.com/github/afroman32/DS-Unit-2-Applied-Modeling/blob/master/module2-wrangle-ml-datasets/wrangle-ml-datasets-assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Inspect

In [0]:
import pandas as pd
import numpy as np

In [0]:
#need the encoding set to "ISO-8859-1" so it will read the weird characters properly 
movies = pd.read_csv('https://raw.githubusercontent.com/afroman32/Unit-2-Build/master/movies.csv',
                     encoding = "ISO-8859-1")

In [4]:
print(movies.shape)
movies.head()

(6820, 15)


,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year
0,8000000.0,Columbia Pictures Corporation,USA,Rob Reiner,Adventure,52287414.0,Stand by Me,R,1986-08-22,89,8.1,Wil Wheaton,299174,Stephen King,1986
1,6000000.0,Paramount Pictures,USA,John Hughes,Comedy,70136369.0,Ferris Bueller's Day Off,PG-13,1986-06-11,103,7.8,Matthew Broderick,264740,John Hughes,1986
2,15000000.0,Paramount Pictures,USA,Tony Scott,Action,179800601.0,Top Gun,PG,1986-05-16,110,6.9,Tom Cruise,236909,Jim Cash,1986
3,18500000.0,Twentieth Century Fox Film Corporation,USA,James Cameron,Action,85160248.0,Aliens,R,1986-07-18,137,8.4,Sigourney Weaver,540152,James Cameron,1986
4,9000000.0,Walt Disney Pictures,USA,Randal Kleiser,Adventure,18564613.0,Flight of the Navigator,PG,1986-08-01,90,6.9,Joey Cramer,36636,Mark H. Baker,1986


In [5]:
movies['score'].describe()

count    6820.000000
mean        6.374897
std         1.003142
min         1.500000
25%         5.800000
50%         6.400000
75%         7.100000
max         9.300000
Name: score, dtype: float64

#Wrangle

In [0]:
# director track record, movies success by director
# make sure to train on features that would come before the target

In [7]:
# number of unique directors
movies['director'].nunique()

2759

In [8]:
# create a list of directors
directors = list(movies['director'])

# remove the duplicates from the list
directors = list(dict.fromkeys(directors))

# check length of directors, should be 2759 to match unique directors
print(len(directors))
type(directors)

2759


list

In [9]:
# test to see average for directors
sub = movies[movies['director'] == movies['director'][3]]
sub['gross'].sum()/len(sub['gross'])

318321163.8333333

In [54]:
ave_gross = []

# run through each director and average the movie gross revenue
for i in range(0,len(directors)):

  # create a subset of all directors in index i
  sub = movies[movies['director'] == directors[i]]

  # average the gross revenues of each movie
  average=sub['gross'].sum()/len(sub['gross'])

  # cast float to int to remove decimal points then add it to ave_gross list
  ave_gross.append(int(average))

# check if ave_gross has correct number of values
print(len(ave_gross))
print(directors[i])

2759
Nicole Garcia


In [55]:
sub

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,dir_ave_gross,writer_ave_gross
6815,0.0,Les Productions du TrÃ©sor,France,Nicole Garcia,Drama,37757.0,From the Land of the Moon,R,2017-07-28,120,6.7,Marion Cotillard,2411,Milena Agus,2016,51398395,47640647


In [37]:
movies[movies['director']==directors[-2]]

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,dir_ave_gross,writer_ave_gross
6747,0.0,Borderline Presents,USA,Nicolas Pesce,Drama,25981.0,The Eyes of My Mother,R,2016-12-02,76,6.2,Kika MagalhÃ£es,6947,Nicolas Pesce,2016,10387002,75976178


In [57]:
ave_gross[-1]

37757

In [0]:
# create a dictionary with directors and their average gross
dir_ave = {directors[i]: ave_gross[i] for i in range(len(directors))}

In [59]:
len(dir_ave)

2759

In [60]:
print(directors[0])
res = list(dir_ave.keys()).index('Nicolas Pesce') 
res

Rob Reiner


2757

In [61]:
# pass the dictionary to the movies dataframe
movies['dir_ave_gross']= movies['director'].map(dir_ave)
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,dir_ave_gross,writer_ave_gross
0,0.0,New World Pictures,USA,Harley Cokeliss,Action,6637565.0,Black Moon Rising,R,1986-01-10,100,5.4,Tommy Lee Jones,4045,John Carpenter,1986,4849211,18418741
1,0.0,Longshot,USA,Paul Bartel,Comedy,1096843.0,The Longshot,PG-13,1986-01-17,89,5.4,Tim Conway,457,Tim Conway,1986,1626657,781727
2,1100000.0,Empire Pictures,USA,John Carl Buechler,Comedy,5450815.0,Troll,PG-13,1986-01-17,82,4.3,Michael Moriarty,7569,Ed Naha,1986,12310408,41382841
3,0.0,TriStar Pictures,USA,Sidney J. Furie,Action,24159872.0,Iron Eagle,PG-13,1986-01-17,117,5.3,Louis Gossett Jr.,11304,Kevin Alyn Elders,1986,13214797,28682422
4,0.0,DIC Entertainment,Canada,Bruno Bianchi,Family,2193642.0,Heathcliff: The Movie,G,1986-01-17,73,5.3,Mel Blanc,187,George Gately,1986,2193642,1338264


In [62]:
# number of unique writers
movies['writer'].nunique()

4199

In [63]:
# create a list of writers
writers = list(movies['writer'])

# remove the duplicates from the list
writers = list(dict.fromkeys(writers))

# check length of writers, should be 4199 to match unique writers
print(len(writers))
type(writers)

4199


list

In [64]:
# test to see average for random writer
sub = movies[movies['writer'] == movies['writer'][3]]
sub['gross'].sum()/len(sub['gross'])

12391598.666666666

In [65]:
writer_gross = []

# run through each writers and average the movie gross revenue
for i in range(0,len(writers)):
  # create a subset of all writers in index i
  sub = movies[movies['writer'] == writers[i]]

  # average the gross revenues of each movie
  average=sub['gross'].sum()/len(sub['gross'])

  # cast float to int to remove decimal points then add it to ave_gross list
  writer_gross.append(int(average))

# check if writer_gross has correct number of values, expect 4199
len(writer_gross)

4199

In [0]:
# create a dictionary with writers and their average gross
dir_ave = {writers[i]: writer_gross[i] for i in range(len(writers))}

In [67]:
# pass the dictionary to the movies dataframe
movies['writer_ave_gross']= movies['writer'].map(dir_ave)
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,dir_ave_gross,writer_ave_gross
0,0.0,New World Pictures,USA,Harley Cokeliss,Action,6637565.0,Black Moon Rising,R,1986-01-10,100,5.4,Tommy Lee Jones,4045,John Carpenter,1986,4849211,15583149
1,0.0,Longshot,USA,Paul Bartel,Comedy,1096843.0,The Longshot,PG-13,1986-01-17,89,5.4,Tim Conway,457,Tim Conway,1986,1626657,1096843
2,1100000.0,Empire Pictures,USA,John Carl Buechler,Comedy,5450815.0,Troll,PG-13,1986-01-17,82,4.3,Michael Moriarty,7569,Ed Naha,1986,12310408,5450815
3,0.0,TriStar Pictures,USA,Sidney J. Furie,Action,24159872.0,Iron Eagle,PG-13,1986-01-17,117,5.3,Louis Gossett Jr.,11304,Kevin Alyn Elders,1986,13214797,12391598
4,0.0,DIC Entertainment,Canada,Bruno Bianchi,Family,2193642.0,Heathcliff: The Movie,G,1986-01-17,73,5.3,Mel Blanc,187,George Gately,1986,2193642,2193642


#Train, Val, Test Split 
(60,20,20)

In [0]:
movies.sort_values(by = 'released', inplace = True)

In [69]:
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,dir_ave_gross,writer_ave_gross
0,0.0,New World Pictures,USA,Harley Cokeliss,Action,6637565.0,Black Moon Rising,R,1986-01-10,100,5.4,Tommy Lee Jones,4045,John Carpenter,1986,4849211,15583149
1,0.0,Longshot,USA,Paul Bartel,Comedy,1096843.0,The Longshot,PG-13,1986-01-17,89,5.4,Tim Conway,457,Tim Conway,1986,1626657,1096843
2,1100000.0,Empire Pictures,USA,John Carl Buechler,Comedy,5450815.0,Troll,PG-13,1986-01-17,82,4.3,Michael Moriarty,7569,Ed Naha,1986,12310408,5450815
3,0.0,TriStar Pictures,USA,Sidney J. Furie,Action,24159872.0,Iron Eagle,PG-13,1986-01-17,117,5.3,Louis Gossett Jr.,11304,Kevin Alyn Elders,1986,13214797,12391598
4,0.0,DIC Entertainment,Canada,Bruno Bianchi,Family,2193642.0,Heathcliff: The Movie,G,1986-01-17,73,5.3,Mel Blanc,187,George Gately,1986,2193642,2193642


In [0]:
movies.reset_index(inplace = True, drop  = True)

In [71]:
movies.head()

,budget,company,country,director,genre,gross,name,rating,released,runtime,score,star,votes,writer,year,dir_ave_gross,writer_ave_gross
0,0.0,New World Pictures,USA,Harley Cokeliss,Action,6637565.0,Black Moon Rising,R,1986-01-10,100,5.4,Tommy Lee Jones,4045,John Carpenter,1986,4849211,15583149
1,0.0,Longshot,USA,Paul Bartel,Comedy,1096843.0,The Longshot,PG-13,1986-01-17,89,5.4,Tim Conway,457,Tim Conway,1986,1626657,1096843
2,1100000.0,Empire Pictures,USA,John Carl Buechler,Comedy,5450815.0,Troll,PG-13,1986-01-17,82,4.3,Michael Moriarty,7569,Ed Naha,1986,12310408,5450815
3,0.0,TriStar Pictures,USA,Sidney J. Furie,Action,24159872.0,Iron Eagle,PG-13,1986-01-17,117,5.3,Louis Gossett Jr.,11304,Kevin Alyn Elders,1986,13214797,12391598
4,0.0,DIC Entertainment,Canada,Bruno Bianchi,Family,2193642.0,Heathcliff: The Movie,G,1986-01-17,73,5.3,Mel Blanc,187,George Gately,1986,2193642,2193642


In [0]:
# set end year for training data set
train_lim = movies['year'].min()+19

# set beginning year for test data
test_lim = movies['year'].max()-4

In [0]:
# movies in years less than training end year
train = movies[movies['year']<= train_lim]

# all movies in years greater than training end year
val_temp = movies[movies['year'] > train_lim]

# movies in years less than test year and greater than training end year
val = val_temp[val_temp['year'] < test_lim]

# movies in years greater than test year
test = movies[movies['year']>=test_lim]

In [74]:
train.shape, val.shape, test.shape

((4400, 17), (1320, 17), (1100, 17))

#Assignment 1

##Baseline

In [75]:
average = movies['gross'].mean()
print(f'{average:,.0f}')

33,497,829


In [76]:
from sklearn.metrics import mean_absolute_error

train['gross'].mean()

y_train = train['gross']
y_test = test['gross']
base_guess = y_train.mean()


# test error
y_base_pred = [base_guess] * len(y_test)
mae_test = mean_absolute_error(y_test, y_base_pred)

print(f'Mean Absolute Error: ${mae_test:,.2f}')

Mean Absolute Error: $45,726,662.13


##Basic Model

In [85]:
!pip install category_encoders==2.*

In [0]:
# set target
target = 'gross'

# set features
features = ['genre', 'rating', 'year', 'dir_ave_gross', 'writer_ave_gross']

# set X matrices and y vectors
X_train = train[features]
y_train = train[target]

X_val = val[features]
y_val = val[target]

X_test = test[features]
y_test = test[target]

In [91]:
import category_encoders as ce
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import make_pipeline
# from math import sqrt

pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    # SimpleImputer(strategy = 'mean'),
    StandardScaler(),
    LinearRegression()
)

# Fit on training set
pipeline.fit(X_train, y_train)

# make predictions
y_pred = pipeline.predict(X_test)

# test mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Mean Absolute Error: {mae:,.2f}')

Test Mean Absolute Error: 19,734,155.80
